<a href="https://colab.research.google.com/github/ShawnNevers/Sudoku/blob/main/Sudoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash
!pip install numpy
!pip install pyomo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.8 MB/s eta 0:00:00


In [ ]:
!apt-get install -y -qq glpk-utils


Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [ ]:
import numpy as np
import pyomo.environ as pyo
from collections import defaultdict

In [ ]:
# @title Initiate Sets, Params, and variables

initial_dict = defaultdict(lambda: 0)

model = pyo.AbstractModel()
model.MacroRow = pyo.Set()
model.MacroCol= pyo.Set()
model.row= pyo.Set()
model.column= pyo.Set()
model.num= pyo.Set()

model.initials= pyo.Param(model.MacroRow, model.row, model.MacroCol, model.column)
model.correct_values=pyo.Var(model.MacroRow, model.row, model.MacroCol, model.column, model.num, within=pyo.Binary)

range_3 = [i for i in range(3)]
value_tuple_list = []
cell_mapping = {}
result_mapping = {}
for maj_row in range_3:
  for min_row in range_3:
    for maj_col in range_3:
      for min_col in range_3:
        row = maj_row * 3 + min_row
        col = maj_col * 3 + min_col
        value_tuple = tuple((maj_row, min_row, maj_col, min_col))
        value_tuple_list.append((maj_row, min_row, maj_col, min_col))
        cell_mapping[row, col] = (maj_row, min_row, maj_col, min_col)
        result_mapping[maj_row, min_row, maj_col, min_col] = row, col

soduku_data = {
    None: {
    'MacroRow': range_3,
    'row': range_3,
    'MacroCol': range_3,
    'column': range_3,
    'num': [1,2,3,4,5,6,7,8,9],
    'initials': initial_dict
    }
}
soduku_data

{None: {'MacroRow': [0, 1, 2],
  'row': [0, 1, 2],
  'MacroCol': [0, 1, 2],
  'column': [0, 1, 2],
  'num': [1, 2, 3, 4, 5, 6, 7, 8, 9],
  'initials': defaultdict(<function __main__.<lambda>()>, {})}}

In [ ]:
# @title Objective and Constraints

def objective_rule(model):
    return sum(model.correct_values[i,j,k,l,n] for i in model.MacroRow for j in model.MacroCol for k in model.row for l in model.column for n in model.num)
model.minCost = pyo.Objective(rule=objective_rule,sense=pyo.minimize)

# Initial values set
def Input_Rule(model,i,j,k,l,n):
  if model.initials[i,j,k,l] in model.num:
    return model.correct_values[i,j,k,l,model.initials[i,j,k,l]] == 1
  else:
    return pyo.Constraint.Skip
model.InputRule = pyo.Constraint(model.MacroRow, model.row, model.MacroCol, model.column, model.num, rule=Input_Rule)

# 1 num per row
def Row_rule(model,i,k,n):
    return sum(model.correct_values[i, k, j, l, n] for j in model.MacroCol for l in model.column)==1
model.RowRule= pyo.Constraint(model.MacroRow, model.row, model.num, rule=Row_rule)

# 1 num per column
def Col_rule(model,j,l,n):
    return sum(model.correct_values[i, k, j, l, n] for i in model.MacroRow for k in model.row)==1
model.ColumnRule= pyo.Constraint(model.MacroCol, model.column, model.num, rule=Col_rule)

# 1 num per 3x3 box
def Box_rule(model,i,j,n):
    return sum(model.correct_values[i, k, j, l, n] for k in model.row for l in model.column)==1
model.BoxRule= pyo.Constraint(model.MacroRow, model.MacroCol, model.num, rule=Box_rule)

# 1 num per cell
def Cell_rule(model,i,j,k,l):
    return sum(model.correct_values[i, k, j, l, n] for n in model.num)==1
model.CellRule= pyo.Constraint(model.MacroRow, model.MacroCol, model.row, model.column, rule=Cell_rule)



In [ ]:
# @title Calculate Result Function
optimizer= pyo.SolverFactory('glpk')
def create_solve_update(soduku_grid, soduku_data):
  instance= model.create_instance(data = soduku_data)
  optimizer.solve(instance)
  for i in instance.correct_values:
    if instance.correct_values[i].value == 1:
      soduku_grid[result_mapping[i[:4]][0]][result_mapping[i[:4]][1]] = i[4]
  return instance
# create_solve_update(soduku_grid, soduku_data).display()


In [ ]:
# @title Running App


from dash import dcc, html
from dash.dependencies import Input, Output

from dash import Dash, dash_table, dcc, html, Input, Output, callback
import pprint
import numpy as np

def np_to_table(np_array, id_value, editable=False):
  rows, columns = np_array.shape
  table = dash_table.DataTable(
        id=id_value,
        columns=[{
            'id': 'column-{}'.format(i),
            'name': ''.format(i+1)


        } for i in range(columns)],
        data=[
            {'column-{}'.format(col): np_array[row][col] if np_array[row][col] else '' for col in range(columns)}
            for row in range(rows)
        ],

        editable=True,
            style_cell={
            'textAlign': 'center',
            'minWidth': '1px', 'width': '10px', 'maxWidth': '20px',
            'whiteSpace': 'normal'},
            style_table={'overflowX': 'auto', 'width':400},
    )
  return table

soduku_grid = np.zeros((9,9))

app = Dash(__name__)

app.layout = html.Div([
    html.Div('Input Grid'),
    np_to_table(soduku_grid, 'editing-prune-data', True),
    html.Div(id='editing-prune-data-output')
])


@callback(Output('editing-prune-data-output', 'children'),
              Input('editing-prune-data', 'data'))
def display_output(rows):
    pruned_rows = []
    recalculate_result_test_list = []
    for index, row in enumerate(rows):
        # require that all elements in a row are specified
        for col_index, val in enumerate(row.values()):
          input_val = int(val) if val != '' else 0
          soduku_grid[index][col_index] = input_val
          soduku_data_index = cell_mapping[index, col_index]
          soduku_data[None]['initials'][soduku_data_index] = input_val
          recalculate_result_test_list.append(input_val if input_val in range(10) else 0)
    recalculate_result = not all([val == 0 for val in recalculate_result_test_list]) and all([val in range(10) for val in recalculate_result_test_list])
    if recalculate_result:
      create_solve_update(soduku_grid, soduku_data)
    return html.Div([
        html.Div('Result'),
        np_to_table(soduku_grid, 'result') if recalculate_result else html.Div('No Solution')
    ])

app.run()

<IPython.core.display.Javascript object>